# Cohort 10: DMV Rental Price Predictor

## 0. Idea
Words!

## 1. Data Ingestion

## 2. Data Munging & Wrangling

In [ ]:
Drop table if exists property_level_one;
Drop table if exists property_level_two;
Drop table if exists property_level_three;
Drop table if exists property_level_four;

##PULL IN PROPERTY MANAGER NAMES: PROPERTY_PARENT_COMPANY & PROPERTY_PARENT_COMPANY_LOOKUP
CREATE TEMPORARY TABLE property_level_one AS
(
	SELECT
		pn.pid property_id,
		pn.name property_name,
		pa.zip zip_code,
		pt.type property_type
	FROM property_name pn
	JOIN property_type pt ON pt.pid = pn.pid
	JOIN property_address pa on pa.pid = pn.pid);

CREATE INDEX firstindex
ON property_level_one (property_id)
;

CREATE TEMPORARY TABLE property_level_two AS
(
	SELECT
		metro_distance.pid property_id,
		metro_distance.distance metro_distance,
		commuter_distance.distance commuter_distance
	FROM 
		(SELECT
			pid,
			IF(distance != "", min(distance), NULL) as distance
		FROM property_unit_metro_station
		GROUP BY pid
		) metro_distance
	JOIN
		(SELECT
			pid,
			IF(distance != "", min(distance), NULL) as distance
		FROM property_unit_commuter_rail
		GROUP BY pid
		) commuter_distance ON commuter_distance.pid = metro_distance.pid
);

CREATE INDEX secondindex
ON property_level_two (property_id)
;

CREATE TEMPORARY TABLE property_level_three AS
(
	SELECT
		ppet.pid property_id,
		ppet.dogsallowed property_amenities_dogs,
		ppet.catsallowed property_amenities_cats,
		IF(ppcl.parent != "Unknown", ppcl.parent, NULL) as parent_company_name
	FROM property_pets ppet
	JOIN property_address pa on pa.pid = ppet.pid
	JOIN property_parent_company ppc on ppc.pid = ppet.pid
	JOIN property_parent_company_lookup ppcl on ppcl.parentid = ppc.parentid
);

CREATE INDEX thirdindex
ON property_level_three (property_id)
;

CREATE TEMPORARY TABLE property_level_four AS
(
	SELECT
		pd.pid property_id,
		pd.nunit num_units,
		pd.yearbuilt year_built,
		pd.nstory num_stories,
		pf.ac property_amenities_ac,
		pf.washdry property_amenities_washdry,
		pg.pool property_amenities_pool,
		pk.disposal property_amenities_kitchen_disposal,
		pk.dwasher property_amenities_kitchen_dishwasher,
		pp.covpark property_parking_covered,
		pp.garpark property_parking_garage,
		pp.lotpark property_parking_lot,
		pp.stpark property_parking_street	
	FROM property_demographics pd
	JOIN property_features pf on pf.pid = pd.pid
	JOIN property_gym pg on pg.pid = pd.pid
	JOIN property_kitchen pk on pk.pid = pd.pid
	JOIN property_parking pp on pp.pid = pd.pid
);

CREATE INDEX fourthindex
ON property_level_four (property_id)
;

CREATE TABLE analytics.sbk_tt_kcpl_cetsample AS
(
	SELECT 
		pu.unit_id,
		pu.pid,
		pu.beds,
		pu.baths,
		pu.minrent,
		pu.maxrent,
		pu.minsqft,
		pu.maxsqft,
		pp1.property_name,
		pp1.zip_code,
		pp1.property_type,
		pp2.metro_distance,
		pp2.commuter_distance,
		pp3.property_amenities_dogs,
		pp3.property_amenities_cats,
		pp3.parent_company_name,
		pp4.num_units,
		pp4.year_built,
		pp4.num_stories,
		pp4.property_amenities_ac,
		pp4.property_amenities_washdry,
		pp4.property_amenities_pool,
		pp4.property_amenities_kitchen_disposal,
		pp4.property_amenities_kitchen_dishwasher,
		pp4.property_parking_covered,
		pp4.property_parking_garage,
		pp4.property_parking_lot,
		pp4.property_parking_street	
	FROM property_units pu 
	JOIN property_level_one pp1 on pu.pid = pp1.property_id
	JOIN property_level_two pp2 on pu.pid = pp2.property_id
	JOIN property_level_three pp3 on pu.pid = pp3.property_id
	JOIN property_level_four pp4 on pu.pid = pp4.property_id)

###DEMO TABLE##
CREATE TABLE demographic AS 
(
SELECT
	edu.zip,
	agesex.pop,
	agesex.pop_m,
	agesex.pop_f,
	pop_18_24_less_than_high,
	pop_18_24_high_school_grad,
	pop_18_24_some_college_associates,
	pop_18_24_bachelors_or_higher,
	pop_25_older_less_than_9th,
	pop_25_older_high_school_no_diploma,
	pop_25_older_high_school_grad,
	pop_25_older_some_college_no_degree,
	pop_25_older_associates_degree,
	pop_25_older_bachelors_degree,
	pop_25_older_grad_prof_degree,
	P_INSURED,
	P_UNINSURED,
	P_INSURED_65UP,
	P_UNINSURED_65UP,
	P_INSURED_M,
	P_UNINSURED_M,
	P_INSURED_F,
	P_UNINSURED_F,
	P_INC_HH,
	P_INC_HH_10,
	P_INC_HH_10_15,
	P_INC_HH_15_25,
	P_INC_HH_25_35,
	P_INC_HH_35_45,
	P_INC_HH_50_75,
	P_INC_HH_75_100,
	P_INC_HH_100_150,
	P_INC_HH_150_200,
	P_INC_HH_200P,
	INC_HH_MEDIAN,
	INC_HH_MEAN,
	EST_HH,
	P_HH_FAM,
	P_HH_NONFAM,
	EST_HH_SIZE,
	EST_FAM_SIZE,
	EST_HH_POP,
	EST_MARRIED_M,
	EST_MARRIED_F,
	P_EDU,
	P_EDU_9L,
	P_EDU_9_12,
	P_EDU_HS,
	P_EDU_COLLEGE_SOME,
	P_EDU_AD,
	P_EDU_BD,
	P_EDU_GD,
	P_VETS,
	P_DISABILITY,
	P_RESIDE_1L,
	P_NATIVE,
	P_FOREIGN,
	P_FOREIGN_BORN,
	P_LANG_NOT_ENGLISH,
	pop_under_5,
	pop_5_to_9,
	pop_10_to_14,
	pop_15_to_19,
	pop_20_to_24,
	pop_25_to_29,
	pop_30_to_34,
	pop_35_to_39,
	pop_40_to_44,
	pop_45_to_49,
	pop_50_to_54,
	pop_55_to_59,
	pop_60_to_64,
	pop_65_to_69,
	pop_70_to_74,
	pop_75_to_79,
	pop_80_to_84,
	pop_85_older,
	pop_median,
	m_median,
	f_median,
	pop_sex_ratio,
	pop_married,
	pop_widowed,
	pop_divorced,
	pop_separated,
	pop_single,
	m_married,
	m_widowed,
	m_divorced,
	m_separated,
	m_single,
	f_married,
	f_widowed,
	f_divorced,
	f_separated,
	f_single,
	tot_house,
	per_house_rec_SNAP,
	per_house_norec_SNAP,
	per_house_type_fam_married,
	per_house_rec_SNAP_type_fam_married,
	per_house_norec_SNAP_type_fam_married,
	per_house_type_fam_unmarried,
	per_house_rec_SNAP_type_fam_unmarried,
	per_house_norec_SNAP_type_fam_unmarried,
	per_house_type_not_fam,
	per_house_rec_SNAP_type_not_fam,
	per_house_norec_SNAP_type_not_fam,
	per_house_type_w_children,
	per_house_rec_SNAP_type_w_children,
	per_house_norec_SNAP_type_w_children,
	per_house_poverty_below,
	per_house_poverty_at_above,
	pop_white,
	pop_black,
	pop_indig,
	pop_asian,
	pop_hawaii,
	pop_other,
	pop_multi
FROM demographic_education edu
JOIN demographic_healthcare h on edu.zip = h.zip
JOIN demographic_income i on edu.zip = i.zip
JOIN demographic_social s on s.zip = edu.zip
JOIN demographics_age_sex agesex on agesex.zip = edu.zip
JOIN demographics_marital m on m.zip = edu.zip
JOIN demographics_poverty p on p.zip = edu.zip
JOIN demographics_race r on r.zip = edu.zip
);

##ALL TOGETHER

## 3. Computation & Analysis

## 4. Modeling & Application

## 5. Reporting & Visualisation